<a href="https://colab.research.google.com/github/goin2crazy/bel_intership/blob/%2Btrain/v2_of_repo_of_gemini_detail_number_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: read pickle files /content/correct_answers (3).pkl and /content/correct_answers (3).pkl and print first 3 items in each

import pickle

with open('/content/correct_answers (3).pkl', 'rb') as f:
  data1 = pickle.load(f)

with open('/content/correct_answers (3).pkl', 'rb') as f:
  data2 = pickle.load(f)

print(data1[:3])
print(data2[:3])


[{'https://auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0406/users/53af29053bf828f96935ed49da214e5aaeb58532/i-img700x700-168602243475b69f364912.jpg': 0, 'https://auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0406/users/53af29053bf828f96935ed49da214e5aaeb58532/i-img700x700-168602243479dvcx364912.jpg': 0, 'https://auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0406/users/53af29053bf828f96935ed49da214e5aaeb58532/i-img700x700-1686022434tdc0fr364912.jpg': 0, 'https://auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0406/users/53af29053bf828f96935ed49da214e5aaeb58532/i-img700x700-1686022434tp7mhy364912.jpg': 1}, {'https://auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0505/users/bfe1efa855e1e8da5e08e0bf6f7ffefe5392ed9e/i-img1200x898-1716798401m6xckr311160.jpg': 0, 'https://auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr000/auc0505/users/bfe1efa855e1e8da5e08e0bf6f7ffefe5392ed9e/i-img1200x898-1716798398ttzwc831

In [3]:
! git clone https://github.com/goin2crazy/bel_intership
%cd /content/bel_intership
! git checkout +train
! pip install -r requirements.txt
%cd /content/

fatal: destination path 'bel_intership' already exists and is not an empty directory.
/content/bel_intership
Already on '+train'
Your branch is up to date with 'origin/+train'.
/content


In [4]:
%cd /content/bel_intership

from picker_model import TargetModel
from dataprocessor import load_data

/content/bel_intership


In [5]:
def prepare_data(data):
  image_links = list()
  labels = list()

  for item in data:
    image_links.extend([k for k, v in item.items()])
    labels.extend([v for k, v in item.items()])

  return image_links, labels

image_links, labels = prepare_data(data1)

In [6]:
import tensorflow as tf
from tensorflow.data import Dataset

p = TargetModel()

/usr/local/lib/python3.10/dist-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


Runtime of __init__: 0.0000 seconds
Runtime of __init__: 2.3699 seconds


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [17]:
from tqdm.notebook import tqdm

def load_img(img):
  link = bytes.decode(img.numpy())
  img = load_data(link)

  return img

def create_dataset(image_links, labels, batch_size=8):
  images = [load_data(l) for l in tqdm(image_links)]

  dataset = Dataset.from_tensor_slices((images,labels))

  dataset = dataset.shuffle(64)
  dataset = dataset.prefetch(64)

  return dataset.batch(batch_size)

ds = create_dataset(image_links[:300], labels[:300], batch_size=128)

  0%|          | 0/300 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Train

In [16]:
# import AdamW optimizer
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='accuracy',
                               patience=25,
                               mode='max',
                               restore_best_weights=True
                               )

model = p.create_model()
model.compile(optimizer=AdamW(learning_rate=1e-6, weight_decay=0.001),
              # binary crossentropy
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(ds, epochs=100)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 237ms/step - accuracy: 0.7833 - loss: 0.2747
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 1.0000 - loss: 0.2574
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - accuracy: 0.8917 - loss: 0.3280
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.7833 - loss: 0.4540
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.8917 - loss: 0.3145
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - accuracy: 0.8917 - loss: 0.1887
Epoch 7/100


KeyboardInterrupt: 